In [1]:
import os
import sys
sys.path.append(os.path.abspath('repo/trans_probs/mvp/'))
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import math
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.interpolate import BSpline
from IPython.display import display, Markdown, Latex
from models import *
import re

In [2]:
exp_name = 'norm1_crossent_comp'
turn = 'b0'

In [3]:
# load model parameters
model_dict = torch.load('models/exps/%s/model_%s.pth.tar' % (exp_name, turn))

In [4]:
# feature dictionary
with open('models/exps/%s/featdict_%s.pickle' % (exp_name, turn), 'rb') as f:
    feat_dict = pickle.load(f)
f.close()
reverse = {}
for col, ind in feat_dict.items():
    reverse[ind] = col

# column list...no idea why I used a dictionary for this tbh
col_list = []
for i in range(len(reverse)):
    next_col = reverse[i]
    col_list.append(next_col)

In [5]:
# class series
class_series = pd.read_csv('models/exps/%s/class_series_%s.csv' % (exp_name, turn), squeeze=True, index_col=0, header=None)
classes = class_series.index.values

In [6]:
net = Cross_comp
num_features = len(col_list)
num_units = 100
num_classes = len(classes)
model = net(num_features, num_units, num_classes)

In [7]:
print('num features: %d' % num_features)

num features: 76


In [8]:
model_dict['fc1.weight'].shape

torch.Size([100, 76])

In [9]:
test_param = model_dict['fc1.weight']
norm = test_param.norm(2, dim=1, keepdim=True)
norm.shape

torch.Size([100, 1])

In [10]:
norm

tensor([[  5.3829],
        [ 13.8804],
        [  5.5796],
        [  2.5494],
        [ 12.7682],
        [  3.0317],
        [  3.5005],
        [  2.5529],
        [  2.3500],
        [ 12.7964],
        [  1.5404],
        [  4.0994],
        [  5.8475],
        [  3.5220],
        [  2.8573],
        [  2.6279],
        [  2.3829],
        [  2.2037],
        [  2.3735],
        [  5.9923],
        [  2.4946],
        [  3.7216],
        [  1.7870],
        [  3.3978],
        [  6.1405],
        [  3.8749],
        [  1.5529],
        [  2.7407],
        [  2.2651],
        [  2.4564],
        [  2.7201],
        [  2.9620],
        [  4.9081],
        [  1.8328],
        [  2.8067],
        [  2.0658],
        [  1.7289],
        [  2.8896],
        [  2.7788],
        [  4.1639],
        [  3.2238],
        [  4.2102],
        [  3.3707],
        [  3.1314],
        [  3.7941],
        [  1.9174],
        [  4.4329],
        [  2.8161],
        [  2.3323],
        [  3.4317],


In [11]:
max_val = 10
desired = torch.clamp(norm, 1,  max_val)

In [12]:
desired

tensor([[  5.3829],
        [ 10.0000],
        [  5.5796],
        [  2.5494],
        [ 10.0000],
        [  3.0317],
        [  3.5005],
        [  2.5529],
        [  2.3500],
        [ 10.0000],
        [  1.5404],
        [  4.0994],
        [  5.8475],
        [  3.5220],
        [  2.8573],
        [  2.6279],
        [  2.3829],
        [  2.2037],
        [  2.3735],
        [  5.9923],
        [  2.4946],
        [  3.7216],
        [  1.7870],
        [  3.3978],
        [  6.1405],
        [  3.8749],
        [  1.5529],
        [  2.7407],
        [  2.2651],
        [  2.4564],
        [  2.7201],
        [  2.9620],
        [  4.9081],
        [  1.8328],
        [  2.8067],
        [  2.0658],
        [  1.7289],
        [  2.8896],
        [  2.7788],
        [  4.1639],
        [  3.2238],
        [  4.2102],
        [  3.3707],
        [  3.1314],
        [  3.7941],
        [  1.9174],
        [  4.4329],
        [  2.8161],
        [  2.3323],
        [  3.4317],


In [14]:
eps = math.pow(10, -8)
param = test_param * (desired / (eps + norm))

In [16]:
param.shape

torch.Size([100, 76])

In [17]:
test = torch.tensor([[1, 2, 3, 4], [4, 3, 2, 1]])

In [19]:
test

tensor([[ 1,  2,  3,  4],
        [ 4,  3,  2,  1]])

In [20]:
tel = torch.tensor([[2], [3]])
tel

tensor([[ 2],
        [ 3]])

In [21]:
test * tel

tensor([[  2,   4,   6,   8],
        [ 12,   9,   6,   3]])